** Duygu analizi:**  Yapacağımız işlem duyguların kategorik olarak düzenlenmesi ve duyguları pozitif, negatif yada nötr olarak sınıflandırılma işlemidir.

In [ ]:
#Öncelikle kullanacağımız python kütüphanaelerini ekliyerek çalışmamıza başlıyoruz
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.model_selection import train_test_split #Bu işlem ile verilerimizi eğitim ve test(%70-%30) olacak çekilde bölüyoruz

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output 

Ben bu çalışmada sadece duygu(sentiment) analizi yapacağım için veri seti içeridinde bulunan ancak ihtiyacım olmayacak sütunları veri setinden çıkarıyorum.

In [ ]:
veri = pd.read_csv('../input/Sentiment.csv')
#Veri setinde sadece ihtiyacım olacak sütunlar kalıyor.
veri = veri[['text', 'sentiment']]


İlk olarak şimdi elimde bulunan veri setini eğitim ve test verileri olacak şekilde ikiye bölüyorum test için %10 ve eğitim için %90 olacak şekilde verileri ayırıyorum, bu çalışmada sadece duyguların pozitif mi yoksa negatifmi olduğu ile ilgileniyorum dolayısı ile nötr duyguları de veri setimden çıkarıyorum.

In [ ]:
#Veri Setimi Eğitim ve test verilerine ayırıyorum
train, test = train_test_split(veri, test_size = 0.1)
#Sonra Veri seti içerisindeki Nötr duyguları çıkarıyorum
train = train[train.sentiment !="Neutral"]

Bir sonraki edım için eğitim setimdeki verilerden Nötr verileri çıkardım dolayısı ile görselleştirme işlemi daha da kolaylaştı. Daha sonra veri seti içerisinde #hashtag olan değerleri ve link içeren değerleri temizliyorum. Şimdi artık en empatik değerleri WordCloud dan  Negatif ve Pozitif olacak şekilde görselleştiriyorum.

In [ ]:
train_pos = train[ train['sentiment'] == 'Positive']
train_pos = train_pos['text']
train_neg = train[ train['sentiment'] == 'Negative']
train_neg = train_neg['text']

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and not word.startswith('#')
                                and word != 'RT'
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
print("Pozirif Kelimeler")
wordcloud_draw(train_pos,'white')
print("Negatif Kelimeler")
wordcloud_draw(train_neg)

Pozitif kelime setinde aşağıdaki kelimeleri ve ifadeleri fark etmek ilginç:
  ** hakikat **, ** güçlü **, ** meşru **, ** birlikte **, ** aşk **, ** iş **
  (**truth**, **strong**, **legitimate**,  **together**, **love**, **job**)

Benim yorumumda, insanlar ideal adaylarının doğru, meşru, iyi ve kötünün üzerinde olduğuna inanmaya eğilimlidirler.
---------------
Aynı şekilde Negatif kelimeler içerisindeki :
**etki**, **haberler**,**asansor müziği**,**hayal kırıklığı**, **yumuşaktop**, **makyaj yapmak**, **Kiraz toplama**, **denemek** gibi kelimelerin olması da ilginç.

Anladığım kadarıyla insanlar kararlı bir şekilde harekete geçmeyi başardılar ve azarlanan adayların çok yumuşak ve vişne almayı düşündüklerini söylediler.

Görselleştirmeden sonra eğitim setinden hashtag, link ve sonlandırıcı kelimeleri çıkardım. 
**Stop Word :** Arama sorgularında kullanılmak üzere önemli bir önem taşımayan kelimelerdir. Genellikle bu kelimeler gereksiz bilgi döndürdüğü için arama sorgularından filtrelenir. (the, for, this vb.)

In [ ]:
tweets =[]
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
                    if 'http' not in word
                    and not word.startswith('@')
                    and not word.startswith('#')
                    and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    tweets.append((words_cleaned, row.sentiment))
    
test_pos = test[test['sentiment'] == 'Positive']
test_pos = test_pos['text']
test_neg = test[test['sentiment'] == 'Negative']
test_neg = test_neg['text']

Bir sonraki adım olarak, nltk lib ile adlandırılan özellikleri ilk önce sık dağılımı ölçüp ve sonuçta oluşan anahtar kelimeleri  çıkardım.

In [ ]:
#Kelimelerin özelliklerini çıkarıyorum.
def get_words_in_tweets(tweets):
    all =[]
    for (words, sentiment) in tweets:
        all.extend(words)
    return all
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['containts(%s)' % word] = (word in document_words)
    return features

En sık tekrar eden kelimeleri görselleştiriyorum. Genellikle kelimeler tartışma merkezi etrafında toplanıyor.

In [ ]:
wordcloud_draw(w_features)

In [ ]:
wordcloud_draw(w_features)

nltk Naive Bayes sınıfandırmacıyı kullanarak kelimeleri özelliklerine göre sınıflandırdım.

In [ ]:
#Naive Bayes sınıflandırıcıyı eğitiyorum
training_set = nltk.classify.apply_features(extract_features, tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)

 Son olarak, çok akıllı olmayan metriklerle, sınıflandırıcı algoritmasının nasıl puanlandırma yaptığını ölçmeye çalıştım.

In [ ]:
neg_cnt = 0
pos_cnt = 0
for obj in test_neg:
    res = classifier.classify(extract_features(obj.split()))
    if(res == 'Negative'):
        neg_cnt = neg_cnt + 1
for obj in test_pos:
    res = classifier.classify(extract_features(obj.split()))
    if(res == 'Positive'):
        pos_cnt = pos_cnt + 1

print('[Negative]: %s/%s' % (len(test_neg), neg_cnt))
print('[Positive]: %s/%s' % (len(test_pos), pos_cnt))

![](http://)## Son değerlendirme.##

Bu projede nltk ve NaiveBayes Machine Learning algoritmasının Sentiment Analizi için ne kadar iyi performans gösterdiğini merak ettim. Benim deneyimime göre, olumsuz yorumlar için oldukça iyi çalışıyor. Sorunlar tweet'ler ironik olduğunda, alaycı referansa sahip veya kendi zor bir içeriğe sahip olduğunda ortaya çıkar.

Aşağıdaki tweet'i düşünün:
* "Merhaba, Liberallerin Trump'u yok edememesi ne kadar üzücü.
Daha önce de düşündüğünüz gibi, ** sad ** ve ** destroy ** ifadeleri, anlam ve bağlamı göz önüne alındığında bu tweet pozitif olsa da, değerlendirmeyi büyük ölçüde etkilemektedir.

Değerlendirme doğruluğunu geliştirmek için, içeriği ve referansları dikkate almak için bir şeye ihtiyacımız var. Proje 2.0 olarak, bir LSTM ağı kurmaya çalışacağım ve sonuçlarını bu nltk Makine Öğrenimi uygulamasına kıyasla kıyaslayacağım. 